<a id="content"></a>
# QF605 Project: Fixed Income Securities
## `Part IV`: Decompounded Options
### Team Members:
>Anirban CHAKRABORTY<br>
Boon Heng GOH<br>
Hanley Mahesh RUPAWALLA<br>
Kantapong ARUNADITYA<br>
Maria Vinitha VIJAYANAND<br>
Retwika HAZRA

- [Question 1](#Question1) <= Click
- [Question 2](#Question2)

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import norm
from scipy import interpolate
from math import log, sqrt, exp
from scipy import integrate
from scipy.integrate import quad

`Importing results from part I (Bootstrapping Swap Curves)`

In [2]:
FS = pd.read_csv('Swap_Rates_Raw.csv', header = 0)  
Discount_Fact = pd.read_csv('Rates_Raw.csv', header = 0)

`Importing results from part II (Swaption Calibration)`

In [3]:
data_SABR = pd.read_csv('SABR_Calibs.csv', header = 0)

`Using Black76 for static replication`

In [4]:
def Black76Lognormal(F, K, r, sigma, T, opt):
    d1 = (log(F/K)+(sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)
    if opt == 'Call':
        return F*exp(-r*T)*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
    elif opt == 'Put':
        return K*exp(-r*T)*norm.cdf(-d2) - F*exp(-r*T)*norm.cdf(-d1)

`Defining SABR model for static replication`

In [5]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

<a id="Question1"></a>
### 1. A decompounded option pays the following at time T = 5y:

$$CMS\ 10y^{\frac{1}{p}} - 0.04^{\frac{1}{q}}$$

where $p$ = 4 and $q$ = 2. Use static replication to value the PV of this payoff.

`Differentiating the function g(F)`

$$g(K) = F^{\frac{1}{4}} - 0.2\ , \ \ \ \ \ \ g'(K) = \frac{1}{4}F^{-\frac{3}{4}}\ , \ \ \ \ \ \ \ g''(K) = -\frac{3}{16}F^{-\frac{7}{4}}$$

In [6]:
def g(K):
    return K**(1/4) - 0.2

def gf(K):
    return (1/4)*(K)**(-3/4)

def gff(K):
    return (-3/16)*(K)**(-7/4)

`IRR is define as:`

$IRR  (K) = \sum_{i = 1}^{N \cdot m} \frac{1}{m} \cdot({1 + \frac{K}{m}})^{-i}$ <br><br>
$IRR' (K) = \sum_{i = 1}^{N \cdot m} (-i)\cdot(\frac{1}{m})^2 \cdot({1 + \frac{K}{m}})^{-i-1}$ <br><br>
$IRR'' (K) = \sum_{i = 1}^{N \cdot m} (-i)\cdot(-i-1) \cdot(\frac{1}{m})^3 \cdot({1 + \frac{K}{m}})^{-i-2}$ <br><br>

where, <br>
&emsp;&emsp;&emsp; delta, $\delta = \frac{1}{m}$

In [7]:
def IRR(K, Tenor, Delta):
    total = 0
    for i in np.arange(1, Tenor/Delta +1):
        total += Delta*(1 + K* Delta)**(-i)
    return total

def IRRf(K, Tenor, Delta):  
    total = 0
    for i in np.arange(1, Tenor/Delta +1):
        total += (-i)*(Delta**2)*(1 + K* Delta)**(-i-1)
    return total

def IRRff(K, Tenor, Delta):
    total = 0
    for i in np.arange(1, Tenor/Delta +1):
        total += (-i)*(-i-1)*(Delta**3)*(1 + K* Delta)**(-i-2)
    return total

`Using quotient rule, 1st & 2nd order derivatives of h(K) are given by,`

$ h(K) = \frac{g(K)}{IRR(K)}$ <br><br>
$ h'(K) = \frac{IRR(K)g'(K)\ \ -\ \ g(K)IRR'(K)}{IRR(K)^2}$<br><br>
$ h''(K) = \frac{IRR(K)g''(K)\ \ - \ IRR''(K)g(K)\ \ - \ 2\cdot IRR'(K)g'(K)}{IRR(K)^2} +\frac{2\cdot IRR'(K)^2g(K)}{IRR(K)^3}\cdot$

In [8]:
def h(K,Tenor,Delta):
    return g(K)/IRR(K, Tenor, Delta)

def hf(K, Tenor, Delta):
    top = IRR(K, Tenor, Delta)*gf(K)-g(K)*IRRf(K, Tenor, Delta)
    bottom = IRR(K, Tenor, Delta)**2
    return top / bottom

def hff(K, Tenor, Delta):
    top = IRR(K,Tenor,Delta)*gff(K)-IRRff(K,Tenor,Delta)*g(K)-2*IRRf(K,Tenor,Delta)*gf(K)
    bottom = IRR(K,Tenor,Delta)**2
    term3 = 2*IRRf(K,Tenor,Delta)**2*g(K)/IRR(K,Tenor,Delta)**3
    return top/bottom + term3

In [9]:
# Forward swap rate - 5y x 10y

F = FS.loc[9, 'Swap_Rate']

In [10]:
# OIS discount factor - D(0, 5y)

D = Discount_Fact.loc[9, 'ON_DF']

`Defining Parameter`

In [11]:
alpha = data_SABR.loc[9, 'alpha'] # this is alpha parameter for 5y x 10y forward swap
beta = 0.9                        # this is pre-determined beta at 0.9
rho = data_SABR.loc[9, 'rho']     # this is rho parameter for 5y x 10y forward swap
nu = data_SABR.loc[9,  'nu']      # this is nu parameter for 5y x 10y forward swap

In [12]:
# Setting Variables

Tenor = 10
Delta = 0.5
T = 5

`PV can be valued as,`

$$ V_0  = D(0,T)g(F) + \int_0^F h''(K)V^{rec}(K)dK + \int_F^\infty h''(K)V^{pay}(K)dK $$

In [13]:
V_rec = integrate.quad(lambda x: hff(x,Tenor,Delta)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Put"),
                                                    0,F)
V_pay = integrate.quad(lambda x: hff(x,Tenor,Delta)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Call"),
                                                    F,1000)

PVoption = D * g(F) + V_rec[0] + V_pay[0]

In [14]:
print("Question 1 - PV of this Payoff is:", PVoption)

Question 1 - PV of this Payoff is: 0.24191109999759403


[Back to top](#content)

<a id=Question2></a>
### 2. Suppose the payoff is now,

$$\left(CMS\ 10y^{\frac{1}{p}} - 0.04^{\frac{1}{q}}\right)^{+}$$

Use static replication to value the PV of this payoff.

`Consider the payoff to be,`

$$g(S_{T}) = S^{\frac{1}{4}} - 0.2 > 0$$

<u>Defining Lower bound limit "L"

$S_{T}^\frac{1}{4} > 0.2$ <br><br>
$S_{T} > 0.2^4$<br><br>
$S_{T} > 0.0016 = L$

In [15]:
L = 0.2**4

`We can value this PV as CMS Caplet and it is given as,`

$$CMS\ \ Caplet = V^{pay}(L)h'(L) + \int^{\infty}_Lh''(K)V^{pay}(K)dK$$

In [16]:
Tenor = 10
Delta = 0.5
T = 5

period1 = hf(L, Tenor, Delta)*Black76Lognormal(F, L, 0, SABR(F, L, T, alpha, 0.9, rho, nu),T,"Call")
period2 = integrate.quad(lambda x: hff(x, Tenor,Delta)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T,"Call"),
                                                    L,1000)
PV_cap = period1+ period2[0]

In [17]:
print("Question 2 - PV of this Payoff is:", PV_cap.round(6))

Question 2 - PV of this Payoff is: 0.029444


[Back to top](#content)